In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from dotenv import load_dotenv
from scipy.stats import skew, kurtosis
from sklearn.model_selection import train_test_split
from optimizers.simulations.order import simulate_from_order_func
from optimizers.utils.cross_validators import vbt_cv_sliding_constructor
from optimizers.pipes.pipe import SQLPipe

load_dotenv()

USER = os.getenv('psql_username')
PASS = os.getenv('psql_password')
DATABASE = 'crypto'
SCHEMA = 'kucoin'

pipe = SQLPipe(SCHEMA, DATABASE, USER, PASS)

In [57]:
assets = ['BTC-USDT', 'ETH-USDT']
slicer = 0 # Slice off first few months of trading to reduce early volatility

df = pipe.query_pairs_trading_backtest(assets)
closes = df.xs('close', level=1, axis=1)[slicer:]
opens = df.xs('open', level=1, axis=1)[slicer:]
assert closes.shape[0] > 8640, 'Less than 1 year of backtesting data present'
assert closes.shape == opens.shape, 'Open and close shape does not match'

print(f"Full data length: {closes.shape[0]}")

train_opens, opens = train_test_split(opens, test_size=0.30, train_size=0.70, shuffle=False)
train_closes, closes = train_test_split(closes, test_size=0.30, train_size=0.70, shuffle=False)
close_folds = vbt_cv_sliding_constructor(train_closes, n_splits=5)
open_folds = vbt_cv_sliding_constructor(train_opens, n_splits=5)

print(f"Test data length: {opens.shape[0]}")
print(f"Train fold length {close_folds[0].shape[0]}")

Full data length: 41672
Test data length: 12502
Train fold length 5834


In [ ]:
from optimizers.simulations.order import simulate_rolling_ou_model
from optimizers.simulations.order import simulate_mult_rolling_ou_model

params = {
    "period": np.arange(1620, 1621, 10, dtype=int),
    "upper_entry": np.arange(1.00, 1.41, 0.05, dtype=float),
    "upper_exit": np.arange(0.75, 0.76, 0.05, dtype=float),
    "lower_entry": np.arange(1.00, 1.41, 0.05, dtype=float) * -1.0,
    "lower_exit": np.arange(0.50, 0.51, 0.05, dtype=float) * -1.0,
}

n = 1
for val in params.values():
    n *= len(val)
print(f'Running {n} simulations')


In [ ]:
pf = simulate_mult_rolling_ou_model(closes, opens, params, order_size=0.50)

In [ ]:
pf.total_return().plot(kind='hist', bins=np.arange(pf.total_return().min(), pf.total_return().max(), 0.01))